<a href="https://colab.research.google.com/github/vatsDivyank/Face-Recoginition-TinyML/blob/main/User_not_User_FineTuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.3



Include Libraries and Dependencies



In [ ]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf

In [ ]:
print(tf.__version__)

2.3.0


Copying the Preprocessing Image from Google Drive to /content


In [ ]:
!cp "/content/drive/MyDrive/Preprocessing_images.py" "/content"

In [ ]:
from Preprocessing_images import DivyankNotDivyank

In [ ]:
MODELS_DIR = os.path.abspath("/content/drive/MyDrive/MODEL_DIR")
MODEL_FILE_PATH_PB= os.path.join(MODELS_DIR, "divyank_not_divyank_FineTuned_Mobilenet.pb")
MODEL_FILE_PATH_FULL_INTEGER= os.path.join(MODELS_DIR, "divyank_not_divyank_FineTuned_Mobilenet_FULL_INTEGER.tflite")
MODEL_FILE_PATH_h5= os.path.join(MODELS_DIR, "divyank_not_divyank_FineTuned_Mobilenet.h5")

MODEL_TFLITE = os.path.join(MODELS_DIR, 'divyank_not_divyank_FineTuned_Mobilenet.tflite')
MODEL_TFLITE_MICRO = os.path.join(MODELS_DIR , 'divyank_not_divyank_Mobilenet.cc')

if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)

LOGS_DIR = os.path.abspath("/content/drive/MyDrive/Logs_Dir/logs")
if not os.path.exists(LOGS_DIR):
    os.mkdir(LOGS_DIR)

MODEL_LOG_DIR = os.path.join(LOGS_DIR , "divyank_not_divyank")

Here, we try to create a simple Model and follow the process of training, testing, and converting the Model to full_integer_quantized version and make it work on Microcontroller.

Definition of an own model, which can be trained for face recoginition.

In [ ]:
IMAGENET_SIZE = 128
IMAGENET_DEPTH = 3
IMAGENET_SHAPE = (IMAGENET_SIZE, IMAGENET_SIZE,IMAGENET_DEPTH)

def build_model(img_shape, num_classes) -> Model:
    base_model = MobileNet(
        input_shape = IMAGENET_SHAPE,
        alpha = 0.25,
        #depth_multiplier = 0.5, #Only for MobileNet
        include_top=False,
        weights="imagenet",
        pooling =max
    )

    num_layers = len(base_model.layers)
    print(f"Number of layers in the base model: {num_layers}")
    fine_tune_at = num_layers - 86 
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False,
        

    input_img = Input(shape=img_shape)
    x = Rescaling(scale=1./127.5, offset=-1.0)(input_img)
    x = Resizing(height=IMAGENET_SIZE, width=IMAGENET_SIZE)(x)
    x = Concatenate()([x, x, x])
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=num_classes)(x)
    y_pred = Activation("softmax")(x)

    model = Model(
        inputs=[input_img],
        outputs=[y_pred]
    )

    model.summary()



    return model


In [ ]:

# %load_ext tensorboard
# %tensorboard --logdir {MODEL_LOG_DIR}

In [ ]:
tensorboard --inspect --logdir {MODEL_LOG_DIR}

In [ ]:
if __name__ == "__main__":

    data = DivyankNotDivyank()
    data.data_augmentation(augment_size= 1000)
    x_train, x_val_,y_train_, y_val_ = data.get_splitted_train_validation_set()
    x_test , y_test = data.get_test_set()

    img_shape = data.img_shape
    num_classes = data.num_classes


    opt = Adam(learning_rate=0.00001)#7e-4


    # Global params
    epochs = 10
    batch_size = 128



    model = build_model(
        img_shape,
        num_classes,
    )
    opt = Adam(learning_rate=0.00001)#7e-4
    
    
    # tb_callback = TensorBoard(
    #     log_dir=MODEL_LOG_DIR,
    #     histogram_freq=1,
    #     write_graph = True
    # )
  
    model.compile(
    loss="categorical_crossentropy",
    optimizer=opt,
    metrics=["accuracy"]
    )

    model.fit(
        x=x_train_ ,
        y= y_train_,
        verbose=1,
        epochs=epochs,
       # callbacks=[tb_callback],
        validation_data=(x_val_, y_val_),
    )
    model.save(MODEL_FILE_PATH_PB)
    model.save(MODEL_FILE_PATH_h5)

NameError: ignored

In [ ]:
score = model.evaluate(
    x_test, y_test,
    verbose = 0,
    batch_size = batch_size
)
print(score)

[1.2733690738677979, 0.7791932225227356]


In [ ]:

#Converting the Model to Tensorflow Lite format with float16 quantization
converter_float16 = tf.lite.TFLiteConverter.from_saved_model(MODEL_FILE_PATH_PB)
converter_float16.optimizations = [tf.lite.Optimize.DEFAULT]
converter_float16.target_spec.supported_types = [tf.float16]
tflite_model_float16 = converter_float16.convert()

#Save to disk
model_float16 = (open("model_quant16.tflite","wb").write(tflite_model_float16))/1024
print("float16 Model size %d KB" % model_float16)

#Converting the Model to the Tensorflow Lite format with dynamic range Quantization
converter_dynamic = tf.lite.TFLiteConverter.from_saved_model(MODEL_FILE_PATH_PB)
converter_dynamic.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_dynamic =  converter_dynamic.convert()

#Save to Disk
model_dynamic_size = (open("model_dynamic.tflite","wb").write(tflite_model_dynamic))/1024
print("Dynamic Model size %d KB" % model_dynamic_size)


#Converting the Model into Tensorflow Lite format with full Interger quantization

#This Quantization requires creation of representative Dataset
images = tf.cast(x_test, tf.float32)
test_data = tf.data.Dataset.from_tensor_slices(images).batch(1)
def representative_dataset_gen():
  for input in test_data.take(100):
    yield[input]

converter_full_integer = tf.lite.TFLiteConverter.from_saved_model(MODEL_FILE_PATH_PB)
converter_full_integer.optimizations = [tf.lite.Optimize.DEFAULT]

converter_full_integer.representative_dataset = representative_dataset_gen
converter_full_integer.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_full_integer.inference_input_type = tf.int8
converter_full_integer.inference_output_type = tf.int8 ## Check hier with tf.int8
tflite_model_full_integer = converter_full_integer.convert()

# Save to disk
model_full_integer=(open("model_full_integer.tflite", "wb").write(tflite_model_full_integer))/ 1024
print("Full Integer Model Size %d KB" % model_full_integer)

# float16 Model size 429 KB
# Dynamic Model size 284 KB
# Full Integer Model Size 315 KB

float16 Model size 106 KB
Dynamic Model size 55 KB
Full Integer Model Size 55 KB


In [ ]:
interpreter = tf.lite.Interpreter(model_path='/content/model_full_integer.tflite')
input_type = interpreter.get_input_details()[0]['dtype']
print('input:', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output:', output_type)

input: <class 'numpy.int8'>
output: <class 'numpy.int8'>


In [ ]:
# Helper function to run the inference on a TFLite Model
def run_tflite_model(tflite_file, test_image_indices):
  global test_images

  #Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path='/content/model_full_integer.tflite')
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate (test_image_indices):
    test_image = test_images[test_image_index]
    test_label = test_labels[test_image_index]

    #Checking if the input data is Quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point
    
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()
  return predictions

In [ ]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file
!xxd -i "model_full_integer.tflite" > {MODEL_TFLITE_MICRO}

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
!cat {MODEL_TFLITE_MICRO}

In [ ]:
  model.load_weights (MODEL_FILE_PATH_h5)
  TEST_PHOTO_DIR = os.path.abspath("/content/drive/MyDrive/Folder_to_test_photos")
  image_names = [f for f in os.listdir(TEST_PHOTO_DIR) if ".jpg"in f or".jpeg" in f or ".png" in f]
  class_names = ["Divyank", "not_Divyank"]
  for image_file_name in image_names:
        image_file_path = os.path.join(TEST_PHOTO_DIR, image_file_name)
        print(image_file_path)
        x = data.load_and_preprocess_custom_image(image_file_path)
        x = np.expand_dims(x, axis=0)
        y_pred = model.predict(x)[0]
        y_pred_class_idx = np.argmax(y_pred)
        y_pred_prob = y_pred[ y_pred_class_idx]
        y_pred_class_name=class_names[y_pred_class_idx]
        plt.imshow(x.reshape(96, 96, 1))
        plt.title(f"Pred class:{y_pred_class_name}, Prob:{y_pred_prob}")
        plt.show()

/content/drive/MyDrive/Folder_to_test_photos/1 (22).jpg


TypeError: ignored